In [2]:
import math
import random

In [3]:
%run partial_words.py
%run core.p

In [7]:
def choose_words(nm):
    return random.choices(sbst,k=nm)



In [12]:
len(choose_words(5000))

5000

In [ ]:
sbst

In [ ]:
spl_Tree = {}

In [ ]:
def messup_words(wrd):
    return wrd
def get_network(nt_nm):
    return None
def load_network(nt_nm):
    return None

In [8]:
# rtTreeName contains the name of the network to load 
# avWords contains the list of available words.
# wdDic contains the chosen words root network choice
# maxDepth
def train_and_choose(rtTreeName,avWords,maxDepth):
    '''
    1. Choose N words from avWords min(300,avWords.count)
    2. Create a new instance of get_network with rtTreeName
    3. For the chosen words do the following
        A. find out the category 1 through (category_count) of the words
        B. Now for train_batch count modify all the words using the initially selected
           word as the target.
    4. Now for all words in avWords assign them to on of the (categor_count) categories
    5. Save the network.
    6. If maxDepth <= 1 return..
    7. For 1 to (cagtegory_count) number of categories:
            maxDepth = maxDepth - 1
            A. Select all the elements of chosen category from the avWords.filter_cat(category)
               and append them to a new instance of avWords
            B. If there is more then one element in a category...
               B1. Recursively call train_and_choose with new avWords category and the sub network
                   name netname_ + category_number
            
    '''
    pass

def spell_word(word,nd,wrList):
    '''
       NOT FINISHED...
       1. Load the network name.
       2. Find category for the word in the network.
       3. If wrList.has_more_one(category)
            with spell_word with word and netname_ + category_number
          elif wrList.has_one(category):
            
           
    '''
    pass

In [13]:
60*50*10

30000

In [75]:
wrl = WordManage()

In [76]:
wrl.set_words(sbst,0)

In [77]:
wrl.all_words()

['people',
 'history',
 'way',
 'art',
 'world',
 'information',
 'map',
 'two',
 'family',
 'government',
 'health',
 'system',
 'computer',
 'meat',
 'year',
 'thanks',
 'music',
 'person',
 'reading',
 'method',
 'data',
 'food',
 'understanding',
 'theory',
 'law',
 'bird',
 'literature',
 'problem',
 'software',
 'control',
 'knowledge',
 'power',
 'ability',
 'economics',
 'love',
 'internet',
 'television',
 'science',
 'library',
 'nature',
 'fact',
 'product',
 'idea',
 'temperature',
 'investment',
 'area',
 'society',
 'activity',
 'story',
 'industry',
 'media',
 'thing',
 'oven',
 'community',
 'definition',
 'safety',
 'quality',
 'development',
 'language',
 'management',
 'player',
 'variety',
 'video',
 'week',
 'security',
 'country',
 'exam',
 'movie',
 'organization',
 'equipment',
 'physics',
 'analysis',
 'policy',
 'series',
 'thought',
 'basis',
 'boyfriend',
 'direction',
 'strategy',
 'technology',
 'army',
 'camera',
 'freedom',
 'paper',
 'environment',
 'ch

In [57]:
wrl.choose_words(20)

['president',
 'horse',
 'taste',
 'lab',
 'thanks',
 'rent',
 'pin',
 'front',
 'particular',
 'effective',
 'patient',
 'assistance',
 'cookie',
 'maintenance',
 'mission',
 'quote',
 'document',
 'model',
 'bug',
 'market']

In [24]:
random.sample([2,3,4,5,6,9],3)

[2, 9, 3]

In [59]:
wrl.set_category('bike',2)
wrl.set_category('cause',2)
wrl.set_category('clerk',2)

In [61]:
l = wrl.filter_list(2)

In [65]:
l.choose_words(3)

['clerk', 'cause', 'bike']